<a href="https://colab.research.google.com/github/abduljunaid02/LLMs/blob/main/makemore_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [23]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [32]:


chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [33]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [35]:
  X.shape, Y.shape, X.dtype, Y.dtype

(torch.Size([32, 3]), torch.Size([32]), torch.int64, torch.int64)

In [38]:
C = torch.randn(27,2)
# C is our way of representing the 27 characters in 2 dimensional space.
#We will allocate the 27 characters to these 27-2 dimenstional embeddings now

In [41]:
#Direct allocation like C[5] is same as doing one hot encoding for 5 and then multiplying it with 5th entry in C
#F.one_hot(torch.tensor(5),num_classes=27).float() @ C

C[5]

tensor([0.8390, 0.6557])

In [44]:
#Accessing works for multiple characters as well

C[torch.tensor([5,6,7])]

tensor([[ 0.8390,  0.6557],
        [ 0.0779,  0.7256],
        [ 0.5928, -0.4863]])

In [49]:

C[X].shape

#X is our dataset. We can embedd the whole of X in C with this C[X] it returns a (32,3) i.e., of X and a 2 dimension embedding of C
#If C were a 5 dimensional embedding it would have been (32,3,5)

torch.Size([32, 3, 2])

In [50]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [51]:
#Create weights and biases

w1 = torch.rand(6, 100) #6 because we are using 3 characters represented in 2 dimensions to predict next character. 3*2 = 6 and 100 neurons
b1 = torch.rand(100)

In [55]:
#It is difficult to multiply a (32,3,2) i.e., our embedding vector with our weights that are (6,100). Matrix mul not possible
#So convert emb to (32,6)

h = torch.tanh(emb.view(-1,6) @ w1 + b1) # using -1 lets torch infer what must come in its place
h.shape

torch.Size([32, 100])

In [56]:
#Creating w2 and b2

w2 = torch.randn(100,27) # (100,27) because 100 comes from prev layers and output is 27 characters
b2 = torch.rand(27)

In [57]:
logits = h @ w2 + b2

In [68]:
logits.shape

torch.Size([32, 27])

In [64]:
#We have 27 outputs and we make it into the probabilities
counts = logits.exp()
prob = counts / counts.sum(dim=1, keepdims=True)
prob.shape

torch.Size([32, 27])

In [69]:
prob.shape

torch.Size([32, 27])

In [72]:
prob[torch.arange(32), Y]

#This is advanced indexing where we are getting the probabilities of the numbers (in y like 5,12,11,6) from all the 27 probabilities
#that prob has

tensor([9.3327e-01, 1.0634e-07, 2.2233e-06, 1.9127e-18, 2.4233e-11, 3.4693e-06,
        8.7830e-13, 5.7582e-14, 3.2422e-09, 1.0000e+00, 1.4797e-16, 8.8187e-17,
        5.4755e-11, 6.0565e-11, 1.0753e-07, 6.7565e-16, 4.7666e-29, 6.2563e-13,
        4.8802e-12, 4.8717e-06, 8.9416e-01, 1.5130e-08, 4.2851e-02, 9.4772e-14,
        2.5102e-06, 1.2891e-17, 3.0098e-06, 7.8717e-10, 5.7873e-08, 1.0000e+00,
        6.9596e-15, 2.5424e-16])

In [74]:
#Calculating negative loss likelikhood

loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(22.1805)